# Project 1: Real-Time Social Sentiment Analysis for Stock Prediction

# Introduction & Project Overview

## Purpose
The primary objective of this project is to develop an end-to-end pipeline that leverages real-time social sentiment analysis to predict stock market movements. Specifically, it aims to analyze the sentiment from various online sources to forecast short-term trends in stock prices.

## Selection of Pharma & Defence Stocks
Pharmaceutical and defense sectors were selected due to their sensitivity to public perception and regulatory changes, making them ideal candidates for sentiment-based analysis. These sectors frequently experience significant/volatile price movements influenced by news coverage and social discussions, providing strong datasets for predictive analysis.

## Data Sources Selection

### Reddit
Reddit was chosen due to its popularity, extensive discussion threads, and niche-focused subreddits that specifically cover financial news, investment strategies, and sector-specific discussions (e.g., pharma and defense stocks).

### NEWSAPI
NEWSAPI provides access to diverse, credible, and timely news sources globally. It allows for comprehensive collection of articles relevant to the target companies, enhancing the quality and depth of the sentiment analysis.

### Twitter (Dropped)
Originally, Twitter (X) was intended to be a primary data source. However, due to recent API limitations (restrictive access, high costs, and limitations on the volume of data retrieval), Twitter was dropped to ensure the feasibility, and sustainability of the project.

## Notebook Objectives
This notebook documents the step-by-step processes implemented during each project phase. The current notebook specifically covers **Phase 1: Data Collection**. Each phase is explained and demonstrated through relevant code and outcomes.

---

# Phase 1: Data Collection

## Summary of Accomplishments
- Set up Reddit and NewsAPI scrapers.
- Pulled and saved news and social data for Pfizer, Moderna, Lockheed Martin, and Raytheon.
- Collected historical OHLCV data using `yfinance`, including 7-day and 14-day moving averages.
- Implemented missing data cleaning (`dropna`) and column formatting.
- Updated file paths in the scripts to use relative paths, ensuring that the project can be easily moved or cloned to different environments without needing to adjust file paths manually.
- Structed all data into a unified SQLite database using `schema.sql` and `load_data.py`.
- Implemented `.gitignore` and `.env` files to keep API keys and credentials secure.
- Verified contents using DB Browser for SQLite.

## Step 1: Setting up APIs
APIs were configured for Reddit (via PRAW) and NEWSAPI.

## Reddit Scraper
Utilises PRAW to scrape posts from specific finance-related subreddits. This project specifically targets subreddits 'stocks', 'wallstreetbets', 'biotechplays', 'defensestocks', and 'biotech'.

```python
# reddit_scraper.py
import praw
import pandas as pd
from datetime import datetime
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# API credentials from .env file
API_ID = os.getenv('API_ID')
API_SECRET = os.getenv('API_SECRET')
AGENT = os.getenv('AGENT')
USSR = os.getenv('USSR')
PASS = os.getenv('PASS')

# Check if Reddit credentials exist
if not all([API_ID, API_SECRET, AGENT, USSR, PASS]):
    raise ValueError("Missing one or more Reddit API credentials in .env file!")

reddit = praw.Reddit(
    client_id=API_ID,
    client_secret=API_SECRET,
    user_agent=AGENT,
    username=USSR,
    password=PASS
                     
# Extract relevant information from subreddits
def scrape_reddit(subreddits, post_limit=50):
    posts = []
    for sub in subreddits:
        subreddit = reddit.subreddit(sub)
        for post in subreddit.hot(limit=post_limit):
            posts.append({
                'subreddit': sub,
                'title': post.title,
                'body': post.selftext,
                'score': post.score,
                'comments': post.num_comments,
                'created_at': datetime.fromtimestamp(post.created_utc)
            })
    return pd.DataFrame(posts)

# Main execution
if __name__ == "__main__":
    try:
        target_subreddits = ['stocks', 'wallstreetbets', 'investing', 'biotechplays', 'defensestocks', 'biotech']
        df = scrape_reddit(target_subreddits)

        if not df.empty:
            print(df.head())

            data_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "..", "data")
            os.makedirs(data_dir, exist_ok=True)
            output_path = os.path.join(data_dir, "reddit_posts.csv")

            df.to_csv(output_path, index=False)
            print(f"Saved scraped posts to {output_path}")
        else:
            print("No posts fetched. Check credentials or subreddit accessibility.")

    except Exception as e:
        print(f"Error: {e}")

    input("Press Enter to exit...")
  
```

## NEWSAPI Scraper
Using NEWSAPI to collect news articles relevant to selected companies. This project specifically collects articles regarding Pfizer, Moderna, Lockheed Martin, and Raytheon.

```python
import requests
import pandas as pd
from datetime import datetime
import time
import os

API_KEY = os.getenv('API_KEY')
BASE_URL = os.getenv('BASE_URL')

# Check if NewsAPI credentials exist
if not all([API_KEY, BASE_URL]):
    raise ValueError("Missing one or both NewsAPI credentials in .env file!")

# Fetches news article
def fetch_news(company_name):
    params = {
        'q': company_name,          # Keyword to search for
        'language': 'en',           
        'pageSize': 100,            # Max n of results per page
        'sortBy': 'publishedAt',    
        'apiKey': API_KEY
    }

    r = requests.get(BASE_URL, params=params)
    data = r.json()

    # Extract relevant info from articles
    articles = []
    for article in data.get('articles', []):
        articles.append({
            'company': company_name,
            'title': article['title'],
            'description': article['description'],
            'published_at': article['publishedAt'],
            'source': article['source']['name'],
            'url': article['url'],
            'scraped_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        })

    return pd.DataFrame(articles)

# Main execution block
if __name__ == "__main__":
    companies = ['Pfizer', 'Moderna', 'Lockheed Martin', 'Raytheon']
    all_news = []

    for company in companies:
        print(f"Fetching news for {company}...")
        df = fetch_news(company)
        all_news.append(df)
        time.sleep(1.5) # Small delay for API

    result_df = pd.concat(all_news, ignore_index=True)

     # Get the directory where the current script is located (data folder)
    script_dir = os.path.dirname(os.path.abspath(__file__))
    data_dir = os.path.join(script_dir, "..", "data")
    output_path = os.path.join(data_dir, "news_articles.csv")

    result_df.to_csv(output_path, index=False)
    print(f"News saved to {output_path}")
```

## Collecting Financial Data (Stock Prices)
To complete the data collection phase, I retrieved historical stock price data for the monitored companies, using the 'yfinance' library, pulling OHLC, Volume, and Date data from Yahoo Finance.

```python
# stock_fetcher.py

import yfinance as yf
import pandas as pd
import os  # Import the os module

# Define target stock tickers and company names
companies = {
    'PFE': 'Pfizer',
    'MRNA': 'Moderna',
    'LMT': 'Lockheed Martin',
    'RTX': 'Raytheon'
}

# Date range for historical stock data
start_date = '2022-01-01'
end_date = '2024-12-31'

# Store all company data in a list
all_data = []

for ticker, name in companies.items():
    print(f" Fetching data for {name} ({ticker})...")

    # Download historical stock data
    stock = yf.download(ticker, start=start_date, end=end_date)
    stock.reset_index(inplace=True)

    # Filter necessary columns and missing data
    stock = stock[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
    stock.dropna(inplace=True)

    # Add moving averages, ticker column, and rearrange
    stock['MA_7'] = stock['Close'].rolling(window=7).mean()
    stock['MA_14'] = stock['Close'].rolling(window=14).mean()
    stock['ticker'] = ticker
    stock = stock[['ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'MA_7', 'MA_14', 'Volume']]

    # Rename columns to match database schema format (lowercase)
    stock.columns = ['ticker', 'date', 'open', 'high', 'low', 'close', 'ma_7', 'ma_14', 'volume']

    all_data.append(stock)

# Concatenate all company DataFrames
df = pd.concat(all_data, ignore_index=True)

# Get the directory where the current script is located, and creates path for .csv
script_dir = os.path.dirname(os.path.abspath(__file__))
data_dir = os.path.join(script_dir, "..", "data")
output_path = os.path.join(data_dir, "stock_data.csv")

# Save the combined DataFrame to a CSV file
df.to_csv(output_path, index=False)
print(f"Stock data (with moving averages) successfully saved to {output_path}")

```

## Step 2: Database Setup (SQLite)
Creating structured database tables to store collected data.

The following tables are designed:
- `reddit_posts`: Contains scraped Reddit content
- `news_articles`: Contains article metadata from NEWSAPI
- `stock_prices`: Contains historical OHLCV stock data & MAs for the selected companies

```python
# database.py
import sqlite3
import os

def create_database_from_schema(schema_file, db_file):
    """
    Creates an SQLite database from a given schema file.

    Args:
        schema_file (str): The path to the schema.sql file.
        db_file (str): The name of the database file to create.
    """
    try:
        with sqlite3.connect(db_file) as conn:
            with open(schema_file, 'r') as f:
                conn.executescript(f.read())
            print(f"Database file '{db_file}' created successfully from schema file '{schema_file}'.")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    except FileNotFoundError:
        print(f"Error: Schema file '{schema_file}' not found.")

if __name__ == "__main__":
    script_dir = os.path.dirname(os.path.abspath(__file__))
    schema_file = os.path.join(script_dir, 'schema.sql')
    db_file = os.path.join(os.path.dirname(script_dir), 'data', 'sentiment.db')

    create_database_from_schema(schema_file, db_file)
```

## Step 3: Loading CSV Data into Database
Data from the CSV files is loaded into the corresponding database tables.

```python
# load_data.py
import sqlite3
import pandas as pd
import os

def load_csv_to_db(csv_file, table_name):
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Connect to the SQLite database
    conn = sqlite3.connect(os.path.join("data", "sentiment.db"))

    # Write the DataFrame to the SQL table
    df.to_sql(table_name, conn, if_exists='append', index=False)

    # Close the connection
    conn.close()
    print(f"Loaded data into {table_name} from {csv_file}")

# Load data from CSV files into the database
load_csv_to_db(os.path.join("data", "reddit_posts.csv"), "reddit_posts")
load_csv_to_db(os.path.join("data", "news_articles.csv"), "news_articles")
load_csv_to_db(os.path.join("data", "stock_data.csv"), 'stock_prices')
```
## Final Note on Phase 1
All of the data gathered in Phase 1 is successfully sorted and centralised in `sentiment.db`, which was inspected using DB Browser.